In [ ]:
!pip install pdfplumber openpyxl

In [ ]:
import re
import pdfplumber
import pandas as pd
from collections import namedtuple
from openpyxl import Workbook

# Adjusted named tuple to include relevant fields
Line = namedtuple('Line', 'sr_no air first_name middle_name last_name form_no marks category sex quota selection_details')

# Adjust the regular expression based on the PDF structure
line_re = re.compile(r'^(\d+)\s+(\d+)\s+(.*?)\s+(\d+)\s+(\d+)\s+(\w+)\s+(M|F)\s+(\S+)\s+(.*)$')

file = 'SelectionList R5 AHU - Only Selection - 8nov.pdf'
lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text()
        for line in text.split('\n'):
            if "Choice Not Available" in line or "Disqualified-Allotted by State-MBBS" in line:
                continue  # Skip lines with "Choice Not Available" or "Disqualified-Allotted by State-MBBS"
            
            match = line_re.match(line)
            if match:
                sr_no, air, name, form_no, marks, category, sex, quota, selection_details = match.groups()

                # Split the name into parts
                name_parts = name.split()
                if len(name_parts) == 2:
                    first_name = name_parts[0]
                    middle_name = ''
                    last_name = name_parts[1]
                elif len(name_parts) == 3:
                    first_name = name_parts[0]
                    middle_name = name_parts[1]
                    last_name = name_parts[2]
                else:
                    first_name = name_parts[0]
                    middle_name = ''
                    last_name = ' '.join(name_parts[1:])

                # Check if the quota starts with a numerical value and move it to selection_details if true
                if re.match(r'^\d', quota):
                    selection_details = quota + ' ' + selection_details
                    quota = ''

                lines.append(Line(sr_no, air, first_name, middle_name, last_name, form_no, marks, category, sex, quota, selection_details))
# Create a pandas DataFrame
df = pd.DataFrame(lines)

# Save the DataFrame to an Excel file
df.to_csv('output1.csv', index=False)


In [ ]:
import pandas as pd
import re

# Load the CSV file
df = pd.read_csv('output.csv')

# Function to extract text before numbers
def extract_text_before_numbers(value):
    match = re.match(r"([^\d]+)", value)
    return match.group(0).strip() if match else ''

# Apply the function to the 'cod_college' column
df['new_column'] = df['selection_details'].apply(extract_text_before_numbers)

# Function to remove text before numbers in 'cod_college'
def remove_text_before_numbers(value):
    match = re.search(r"\d.*", value)  # Find the first occurrence of a digit and everything after it
    return match.group(0).strip() if match else value  # Return the matched part or the original value if no match

# Apply the function to the 'cod_college' column
df['selection_details'] = df['selection_details'].apply(remove_text_before_numbers)

# Merge 'Quota' and 'new_column' into a single column
df['Quota'] = df['category'].fillna('') + ' ' + df['quota'].fillna('') + df['new_column'].fillna('')

# Save the updated DataFrame to a new CSV file
print(df)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False)


In [ ]:
# Group by college, category, and gender to find  max AIR
max_air_df = df.loc[df.groupby(['code_college', 'category', 'sex','Quota'])['air'].idxmax()]

# Rename columns to indicate max
max_air_df = max_air_df.rename(columns={'air': 'max_air', 'sr_no': 'max_sr_no', 'neet_roll_no': 'max_neet_roll_no', 'cet_form_no': 'max_cet_form_no', 'Quota': 'max_quota'})

# Merge the min and max DataFrames on college, category, and gender
result_df = pd.merge(min_air_df, max_air_df, on=['code_college', 'category', 'sex','Quota', suffixes=( '_max'))

# Add the Course and Cap_round_No columns with specified values
result_df['Course'] = '#add course name '
result_df['Cap_round_No'] = 2

# Select relevant columns for the final output
columns_to_keep = [
    'code_college', 'category', 'sex',
    'max_air',  'max_quota',
]

# Display the result
final_df = result_df[columns_to_keep]
final_df

#convert into csv
final_df.to_csv('output_Max.csv', index=False)